In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [2]:
# 1. Создаем сессию Spark
spark = SparkSession.builder \
    .appName("Avtovaz") \
    .getOrCreate()

In [3]:
# 2. Загружаем данные из CSV файла
df = spark.read.option("header", True).option("sep", ",").csv("avtovaz.csv")

In [5]:
# Просмотр первых 5 строк
df.show(5, truncate=False)

+----+----------------------------------+-------------------------------+-----------------------------------+-----------------------------+---------------------------------+---------------------------------------+--------------------+------------------------------------------------------+-------------------------------------------+------------------------------------------------------+-------------------+----+
|Год |Реализация (внутренний рынок: тыс)|Реализация (внешний рынок: тыс)|Себестоимость реализации (млн. руб)|Валовая прибыль от реализации|Общая сумма дивидендов (млн. руб)|Чистая прибыль отчётного года(млн. руб)|Всего текущие активы|Потоки денежных средств от инвестиционной деятельности|Потоки денежных средств от фин деятельности|Чистые денежные средства  от операционной деятельности| Прибыль до налогов| Тип|
+----+----------------------------------+-------------------------------+-----------------------------------+-----------------------------+---------------------------------

In [6]:
# Просмотр схемы данных
df.printSchema()

root
 |-- Год: string (nullable = true)
 |-- Реализация (внутренний рынок: тыс): string (nullable = true)
 |-- Реализация (внешний рынок: тыс): string (nullable = true)
 |-- Себестоимость реализации (млн. руб): string (nullable = true)
 |-- Валовая прибыль от реализации: string (nullable = true)
 |-- Общая сумма дивидендов (млн. руб): string (nullable = true)
 |-- Чистая прибыль отчётного года(млн. руб): string (nullable = true)
 |-- Всего текущие активы: string (nullable = true)
 |-- Потоки денежных средств от инвестиционной деятельности: string (nullable = true)
 |-- Потоки денежных средств от фин деятельности: string (nullable = true)
 |-- Чистые денежные средства  от операционной деятельности: string (nullable = true)
 |--  Прибыль до налогов: string (nullable = true)
 |--  Тип: string (nullable = true)



In [7]:
# 3. Преобразуем столбцы в нужные типы данных
new_columns_standardized = []
for col_name in df.columns:
# Очистка имени колонки
    cleaned_name = col_name.strip() \
                            .replace(" ", "_") \
                            .replace("(", "_") \
                            .replace(")", "_") \
                            .replace(".", "") \
                            .lower()
# Убираем возможные двойные подчеркивания
    cleaned_name = "_".join(filter(None, cleaned_name.split("_")))
    new_columns_standardized.append(cleaned_name)

# Применяем новые имена к DataFrame
df = df.toDF(*new_columns_standardized)
print(df.columns)

columns_to_cast_types = {
    "год": IntegerType(),
    "реализация_внутренний_рынок:_тыс": DoubleType(),
    "реализация_внешний_рынок:_тыс": DoubleType(),
    "себестоимость_реализации_млн_руб": DoubleType(),
    "валовая_прибыль_от_реализации": DoubleType(),
    "общая_сумма_дивидендов_млн_руб": DoubleType(),
    "чистая_прибыль_отчётного_года_млн_руб": DoubleType(),
    "всего_текущие_активы": DoubleType(),
    "потоки_денежных_средств_от_инвестиционной_деятельности": DoubleType(),
    "потоки_денежных_средств_от_фин_деятельности": DoubleType(),
    "чистые_денежные_средства_от_операционной_деятельности": DoubleType(),
    "прибыль_до_налогов": DoubleType(),
    "тип": StringType()
}

# Применяем преобразования типов
for col_name, col_type in columns_to_cast_types.items():
    df = df.withColumn(col_name, F.col(col_name).cast(col_type))
df.printSchema()

['год', 'реализация_внутренний_рынок:_тыс', 'реализация_внешний_рынок:_тыс', 'себестоимость_реализации_млн_руб', 'валовая_прибыль_от_реализации', 'общая_сумма_дивидендов_млн_руб', 'чистая_прибыль_отчётного_года_млн_руб', 'всего_текущие_активы', 'потоки_денежных_средств_от_инвестиционной_деятельности', 'потоки_денежных_средств_от_фин_деятельности', 'чистые_денежные_средства_от_операционной_деятельности', 'прибыль_до_налогов', 'тип']
root
 |-- год: integer (nullable = true)
 |-- реализация_внутренний_рынок:_тыс: double (nullable = true)
 |-- реализация_внешний_рынок:_тыс: double (nullable = true)
 |-- себестоимость_реализации_млн_руб: double (nullable = true)
 |-- валовая_прибыль_от_реализации: double (nullable = true)
 |-- общая_сумма_дивидендов_млн_руб: double (nullable = true)
 |-- чистая_прибыль_отчётного_года_млн_руб: double (nullable = true)
 |-- всего_текущие_активы: double (nullable = true)
 |-- потоки_денежных_средств_от_инвестиционной_деятельности: double (nullable = true)
 |--

In [8]:
# 4. Анализ динамики чистой прибыли и ранжирование по росту
# Расчет чистой прибыли за предыдущий год
df_task1 = df.withColumn(
    "чистая_прибыль_предыдущего_года",
    F.lag(F.col("чистая_прибыль_отчётного_года_млн_руб"), 1).over(Window.orderBy("год"))
)
# Вычисление изменения чистой прибыли
df_task1 = df_task1.withColumn(
    "изменение_прибыли",
    F.col("чистая_прибыль_отчётного_года_млн_руб") - F.col("чистая_прибыль_предыдущего_года")
)
# Ранжирование годов по абсолютному росту чистой прибыли
window_rank_spec = Window.orderBy(F.desc("изменение_прибыли"))
df_task1 = df_task1.withColumn(
    "ранг_роста_прибыли",
    F.rank().over(window_rank_spec)
)
df_task1.select("год",
            "чистая_прибыль_отчётного_года_млн_руб",
            "чистая_прибыль_предыдущего_года",
            "изменение_прибыли",
            "ранг_роста_прибыли") \
    .orderBy("ранг_роста_прибыли") \
    .show(25,truncate=False)

+----+-------------------------------------+-------------------------------+-----------------+------------------+
|год |чистая_прибыль_отчётного_года_млн_руб|чистая_прибыль_предыдущего_года|изменение_прибыли|ранг_роста_прибыли|
+----+-------------------------------------+-------------------------------+-----------------+------------------+
|2010|1500.0                               |-38500.0                       |40000.0          |1                 |
|2017|-12384.0                             |-35467.0                       |23083.0          |2                 |
|2018|5860.0                               |-12384.0                       |18244.0          |3                 |
|2001|19056.0                              |4554.0                         |14502.0          |4                 |
|2016|-35467.0                             |-43233.0                       |7766.0           |5                 |
|2000|4554.0                               |1021.0                         |3533.0      

In [9]:
# 5. Категоризация годов по уровню чистой прибыли и агрегация 
# Создаем новую колонку категория_прибыли
"""" условие категорий:
Высокоприбыльный год": если чистая прибыль > 5000 млн руб.
"Среднеприбыльный год": если чистая прибыль от 1000 до 5000 млн руб. (включительно)
"Низкоприбыльный год": если чистая прибыль от 0 до 1000 млн руб. (исключая 0)
"Убыточный год": если чистая прибыль < 0
"Нулевая прибыль": если чистая прибыль = 0"""
df_task2 = df.withColumn(
    "категория_прибыли", 
    F.when(F.col("чистая_прибыль_отчётного_года_млн_руб") > 5000, "Высокоприбыльный год")
    .when((F.col("чистая_прибыль_отчётного_года_млн_руб") > 1000) &  (F.col("чистая_прибыль_отчётного_года_млн_руб") <= 5000), "Среднеприбыльный год")
    .when((F.col("чистая_прибыль_отчётного_года_млн_руб") > 0) &  (F.col("чистая_прибыль_отчётного_года_млн_руб") <= 1000), "Низкоприбыльный год")
    .when(F.col("чистая_прибыль_отчётного_года_млн_руб") < 0, "Убыточный год")
    .when(F.col("чистая_прибыль_отчётного_года_млн_руб") == 0, "Нулевая прибыль")
)

df_task2.groupBy(F.col("категория_прибыли")).agg(F.count("год").alias("количество_лет")
                                            ,F.round(F.avg(F.col("реализация_внутренний_рынок:_тыс")
                                                           +F.col("реализация_внешний_рынок:_тыс")),2).alias("средняя_общая_реализация_тыс_шт")
                                            ).orderBy(F.col("количество_лет").desc()).show()

+--------------------+--------------+-------------------------------+
|   категория_прибыли|количество_лет|средняя_общая_реализация_тыс_шт|
+--------------------+--------------+-------------------------------+
|Среднеприбыльный год|            11|                         662.48|
|       Убыточный год|             8|                         412.63|
| Низкоприбыльный год|             3|                         394.17|
|Высокоприбыльный год|             2|                          582.0|
+--------------------+--------------+-------------------------------+

